In [1]:
#RVFL on breast cancer dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load and preprocess data
data = load_breast_cancer()
X = StandardScaler().fit_transform(data.data)
y = np.eye(2)[data.target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter search space
hidden_list = [10, 20, 30, 40]
activations = {"sigmoid": sigmoid, "tanh": tanh}

best_config = {}
best_acc = 0.0
results = []

for act_name, activation in activations.items():
    for n_hidden in hidden_list:
        np.random.seed(42)  # To keep results reproducible
        W = np.random.randn(n_hidden, X.shape[1])
        b = np.random.randn(n_hidden)

        # Compute H
        def compute_H(X):
            H = activation(np.dot(X, W.T) + b)
            return np.hstack([H, X])

        # Train
        start_train = time.time()
        H_train = compute_H(X_train)
        output_weights = np.linalg.pinv(H_train) @ y_train
        end_train = time.time()

        # Predict train
        y_train_pred = H_train @ output_weights
        train_labels = np.argmax(y_train_pred, axis=1)
        train_true = np.argmax(y_train, axis=1)
        train_acc = accuracy_score(train_true, train_labels)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

        # Predict test
        start_test = time.time()
        H_test = compute_H(X_test)
        y_test_pred = H_test @ output_weights
        end_test = time.time()

        test_labels = np.argmax(y_test_pred, axis=1)
        test_true = np.argmax(y_test, axis=1)
        test_acc = accuracy_score(test_true, test_labels)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

        result = {
            'activation': act_name,
            'n_hidden': n_hidden,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'train_time': end_train - start_train,
            'test_time': end_test - start_test
        }

        results.append(result)

        if test_acc > best_acc:
            best_acc = test_acc
            best_config = result

        print(f"[{act_name}] Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f} | TrainRMSE={train_rmse:.4f} TestRMSE={test_rmse:.4f}")

print("\nBest Configuration:")
print(best_config)


[sigmoid] Hidden=10 | TrainAcc=0.9736 TestAcc=0.9649 | TrainRMSE=0.2100 TestRMSE=0.2278
[sigmoid] Hidden=20 | TrainAcc=0.9802 TestAcc=0.9298 | TrainRMSE=0.1971 TestRMSE=0.2338
[sigmoid] Hidden=30 | TrainAcc=0.9868 TestAcc=0.9474 | TrainRMSE=0.1854 TestRMSE=0.2136
[sigmoid] Hidden=40 | TrainAcc=0.9802 TestAcc=0.9561 | TrainRMSE=0.1906 TestRMSE=0.2297
[tanh] Hidden=10 | TrainAcc=0.9626 TestAcc=0.9649 | TrainRMSE=0.3887 TestRMSE=0.4157
[tanh] Hidden=20 | TrainAcc=0.9846 TestAcc=0.9298 | TrainRMSE=0.2610 TestRMSE=0.3207
[tanh] Hidden=30 | TrainAcc=0.9824 TestAcc=0.9474 | TrainRMSE=0.2561 TestRMSE=0.3041
[tanh] Hidden=40 | TrainAcc=0.9780 TestAcc=0.9561 | TrainRMSE=0.2479 TestRMSE=0.2803

Best Configuration:
{'activation': 'sigmoid', 'n_hidden': 10, 'train_acc': 0.9736263736263736, 'test_acc': 0.9649122807017544, 'train_rmse': np.float64(0.20996718209347373), 'test_rmse': np.float64(0.2278382780271637), 'train_time': 0.03893089294433594, 'test_time': 0.00015115737915039062}


In [2]:
#RVFL on MNIST dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load MNIST dataset
print("Loading MNIST dataset...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X = X / 255.0  # Normalize pixel values to [0, 1]
y = y.astype(int)
y_onehot = np.eye(10)[y]  # One-hot encode labels

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42
)

# Hyperparameter settings
hidden_nodes_list = [10, 20, 30]
activation_functions = {'sigmoid': sigmoid, 'tanh': tanh}

best_config = {}
best_test_acc = 0

print("Starting hyperparameter tuning...\n")

for act_name, act_func in activation_functions.items():
    for n_hidden in hidden_nodes_list:
        input_size = X.shape[1]
        W = np.random.randn(n_hidden, input_size)
        b = np.random.randn(n_hidden)

        def compute_H(X):
            H = act_func(np.dot(X, W.T) + b)
            return np.hstack([H, X])  # Concatenate hidden layer output and input

        # Train
        start_train = time.time()
        H_train = compute_H(X_train)
        output_weights = np.linalg.pinv(H_train) @ y_train
        end_train = time.time()

        # Predict on training data
        y_train_pred = H_train @ output_weights
        train_pred_labels = np.argmax(y_train_pred, axis=1)
        train_true_labels = np.argmax(y_train, axis=1)
        train_acc = accuracy_score(train_true_labels, train_pred_labels)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

        # Predict on test data
        start_test = time.time()
        H_test = compute_H(X_test)
        y_test_pred = H_test @ output_weights
        end_test = time.time()

        test_pred_labels = np.argmax(y_test_pred, axis=1)
        test_true_labels = np.argmax(y_test, axis=1)
        test_acc = accuracy_score(test_true_labels, test_pred_labels)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

        print(f"[{act_name}] Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f} | TrainRMSE={train_rmse:.4f} TestRMSE={test_rmse:.4f}")

        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_config = {
                'activation': act_name,
                'n_hidden': n_hidden,
                'train_acc': train_acc,
                'test_acc': test_acc,
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_time': end_train - start_train,
                'test_time': end_test - start_test
            }

# Print best result
print("\nBest Configuration:")
for k, v in best_config.items():
    print(f"{k}: {v}")


Loading MNIST dataset...
Starting hyperparameter tuning...

[sigmoid] Hidden=10 | TrainAcc=0.8616 TestAcc=0.8544 | TrainRMSE=0.1941 TestRMSE=0.1979
[sigmoid] Hidden=20 | TrainAcc=0.8666 TestAcc=0.8581 | TrainRMSE=0.1924 TestRMSE=0.1960
[sigmoid] Hidden=30 | TrainAcc=0.8672 TestAcc=0.8589 | TrainRMSE=0.1919 TestRMSE=0.1956
[tanh] Hidden=10 | TrainAcc=0.8568 TestAcc=0.8506 | TrainRMSE=0.1961 TestRMSE=0.1999
[tanh] Hidden=20 | TrainAcc=0.8616 TestAcc=0.8562 | TrainRMSE=0.1942 TestRMSE=0.1979
[tanh] Hidden=30 | TrainAcc=0.8653 TestAcc=0.8586 | TrainRMSE=0.1926 TestRMSE=0.1962

Best Configuration:
activation: sigmoid
n_hidden: 30
train_acc: 0.8672142857142857
test_acc: 0.8589285714285714
train_rmse: 0.1918564836498676
test_rmse: 0.19559669185177567
train_time: 16.447969436645508
test_time: 0.09923219680786133
